In [1]:
import napari
import os, glob, enum
from skimage.io import imread
from tqdm import tqdm
from natsort import natsorted
import numpy as np
import tifffile as tiff


In [2]:
data_folder = '/run/user/1000/gvfs/smb-share:server=lowe-sn00.biochem.ucl.ac.uk,share=lowegrp/Data/Nathan/kraken/pcna_h2b/MK0000/Pos15'#'/media/nathan/Transcend/Nathan/data/pcna/test_210422/Pos7'
output_dir = os.path.join(data_folder,'stacks')

In [3]:
@enum.unique
class channels(enum.Enum):
    bf = 0 
    gfp = 1
    rfp = 2
    irfp = 3

In [4]:
## compile stacks
if 'stacks' not in os.listdir(data_folder) or os.listdir(os.path.join(data_folder,'stacks')) == []:
    output_dir = os.path.join(data_folder,'stacks')
    print('creating directory', output_dir) 
    os.mkdir(output_dir) if 'stacks' not in os.listdir(data_folder) else print(output_dir,'directory already exists')
    for channel in channels:
        print(channel.name, 'compiling')
        files = glob.glob(os.path.join(data_folder,'img_channel00{}*.tif'.format(channel.value)))
        stack = []
        for file in tqdm(natsorted(files)):
            tif = np.array(tiff.imread(file),dtype=np.uint8)
            tif_w,tif_h = np.shape(tif)[0],np.shape(tif)[1]
            tif = tif[int((tif_w-1200)/2):int(tif_w-(tif_w-1200)/2),int((tif_h-1600)/2):int(tif_h-(tif_h-1600)/2)] ## cropping
            stack.append(tif)
        stack = np.stack(stack,axis=0)
        with tiff.TiffWriter(os.path.join(output_dir,'{}.tif'.format(channel.name)),imagej=True) as output_file:
            output_file.save(stack)
        print(channel.name, 'saved out')
else:
    print('looks like stacks already exist bro')

looks like stacks already exist bro


In [ ]:
tif

In [5]:
### load images
stack_fn = {}
for channel in channels:
    stack_fn[channel.name] = glob.glob(os.path.join(output_dir, '{}.tif'.format(channel.name)))
# gfp = imread(stack_fn['gfp'][0])
# rfp = imread(stack_fn['rfp'][0])
# irfp = imread(stack_fn['irfp'][0])

### load single image
gfp = imread(os.path.join(data_folder,'img_channel001_position015_time000000000_z000.tif'))

In [7]:
### launch napari
with napari.gui_qt():
    viewer = napari.Viewer()
    
    viewer.add_image(gfp, name="gfp", blending = "additive", colormap = "green")
    viewer.add_image(rfp, name="rfp", contrast_limits = (0, 50), blending = "additive", colormap = "magenta")
    viewer.add_image(irfp, name="irfp", blending = "additive", colormap = "cyan")

---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
~/anaconda3/envs/cellpose/lib/python3.7/site-packages/napari/_qt/qt_event_loop.py in gui_qt(startup_logo=False, gui_exceptions=False, force=False)
    248         splash.close()
    249     try:
--> 250         yield app
        app = <PyQt5.QtWidgets.QApplication object at 0x7fadbf561e10>
    251     except Exception:
    252         notification_manager.receive_error(*sys.exc_info())

<ipython-input-7-f6f918a68b0e> in <module>
      4 
      5     viewer.add_image(gfp, name="gfp", blending = "additive", colormap = "green")
----> 6     viewer.add_image(rfp, name="rfp", contrast_limits = (0, 50), blending = "additive", colormap = "magenta")
        global viewer.add_image = <bound method ViewerModel.add_image of Viewer(axes=Axes(visible=False, labels=True, colored=True, dashed=False, arrows=True), camera=Camera(center=(0.0, 675.5, 843.5